# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
! pip install citipy 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import random
from random import sample
import os
import csv
from pprint import pprint

# Import API key
import api_keys
from api_keys import api_key


# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
unique_cities = len(cities)
print(f"From 1500 sets of lats and lngs, there are {unique_cities} unique cities" )


From 1500 sets of lats and lngs, there are 630 unique cities


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [37]:
# Grab 500 random cities from unique cities list above
city_sample = sample(cities,500)
city_sample

['orange',
 'nowy targ',
 'salalah',
 'nome',
 'lazaro cardenas',
 'kaitangata',
 'kargasok',
 'inhambane',
 'fortuna',
 'laives',
 'kroya',
 'chuy',
 'atuona',
 'thinadhoo',
 'astaneh-ye ashrafiyeh',
 'airai',
 'vanimo',
 'seoul',
 'kalmunai',
 'portland',
 'bandarbeyla',
 'aquiraz',
 'henties bay',
 'saint-denis',
 'paamiut',
 'berlevag',
 'marfino',
 'biguacu',
 'pervomayskiy',
 'cazaje',
 'saint-ambroise',
 'saint-philippe',
 'nanortalik',
 'bambanglipuro',
 'formoso do araguaia',
 'burnie',
 'biak',
 'qaanaaq',
 'puerto madero',
 'aykhal',
 'tuatapere',
 'danville',
 'saint paul',
 'maputo',
 'nikolskoye',
 'urengoy',
 'taolanaro',
 'lephepe',
 'charters towers',
 'poltavka',
 'herat',
 'saint george',
 'bluff',
 'hofn',
 'constitucion',
 'grindavik',
 'guerrero negro',
 'mattawa',
 'marv dasht',
 'upernavik',
 'aswan',
 'ahipara',
 'caldas novas',
 'bambous virieux',
 'atocha',
 'luderitz',
 'las vegas',
 'manica',
 'castro',
 'drahovo',
 'pimentel',
 'conakry',
 'hare bay',
 'se

In [40]:
# Perform a weather check on each city using a series of successive API calls.
# Include a print log of each city as it's being processed (with the city number and city name).

city_name = []
city_ID = []
city_lat = []
city_lngs = []
city_temp_F = []
city_humidity = []
city_cloudiness = []
city_windspeed = []
unfoundcities = 0

for city in city_sample:
    
    try:
        base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="
        final_url = f"{base_url}{api_key}&q={city}"
        response = requests.get(final_url).json()
        
        city_name.append(response['name'])
        city_ID.append(response['id'])
        city_lat.append(response['coord']['lat'])
        city_lngs.append(response["coord"]["lon"])
        city_temp_F.append(response["main"]["temp"])
        city_humidity.append(response["main"]["humidity"])
        city_cloudiness.append(response["clouds"]["all"])
        city_windspeed.append(response["wind"]["speed"])

    except:
        unfoundcities += 1
        pass 

print(f"Unfound cities: {unfoundcities}")

Unfound cities: 44


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [44]:
# Display the data frame
foundcities_df = pd.DataFrame({
                    "City": city_name,
                    "ID": city_ID,
                    "Lat": city_lat,
                    "Lon": city_lngs,
                    "Temp": city_temp_F,
                    "Humidity": city_humidity,
                    "Cloudiness": city_cloudiness,
                    "Wind Speed": city_windspeed
    })

foundcities_df.head()


City       ID    Lat     Lon   Temp  Humidity  Cloudiness  \
0           Orange  5379524  33.70 -117.77  78.73        39          40   
1        Nowy Targ   763523  49.48   20.03  44.20        87          75   
2          Salalah   286621  17.02   54.09  73.40        46           0   
3             Nome  5870133  64.50 -165.41  21.92        79           1   
4  Lázaro Cárdenas  4026082  17.96 -102.20  82.26        75           1   

   Wind Speed  
0        4.70  
1        5.82  
2        4.70  
3        4.38  
4        6.35

In [51]:
# Export the dataframe to a csv file and skip the header row
foundcities_df.to_csv("Found Cities.csv", index=False)

print(f"There are {len(foundcities_df)} cities found out of 500 random cities generated.")

There are 456 cities found out of 500 random cities generated.


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot